# Data processing <a name="index" />

1. [example intro](#example intro)
1. [numpy package](#numpy package)
1. [ndarray operations](#operations)
1. [indexing](#indexing)
1. [copies and views](#view)
1. [advanced indexing and broadcasting](#broadcasting)

## Example intro <a name="example intro" />

In [ ]:
import numpy as np
from scipy.stats import gaussian_kde
from bokeh.io import output_notebook, push_notebook, show
from bokeh.plotting import figure
from ipywidgets import interact

output_notebook()

In [ ]:
n_samples, n_vars = 10, 1000
X = np.random.uniform(size=(n_samples, n_vars), low=-1, high=1)
s = X.sum(axis=0)
g = gaussian_kde(s, 1)
x = np.linspace(-n_samples, n_samples, 200 * n_samples + 1)

# set up figure
p = figure(title="kde of sum of {} uniform variables on [-1, 1]".format(n_samples), plot_height=400, plot_width=600)
r = p.line(x, g(x))
h = show(p, notebook_handle=True)

@interact(kde_width=(.01, 1, .01))
def update(kde_width):
    g = gaussian_kde(s, kde_width)
    r.data_source.data['y'] = g(x)
    push_notebook(handle=h)
    

## `numpy` package <a name="numpy package" />
[back to index](#index)

### `ndarray` initialisation and elementary operators

In [ ]:
import numpy as np
x = np.identity(10)              # The identity matrix in IF^{10x10} (IF = IR ou IC)
y = np.ones(shape=(10, 10)) * .1 # a matrix of ones of shape 10x10, all elements multiplied by 0.1
z = y - x                        # difference of two matrices (must be of "compatible" size)
z = np.diag(z.sum(axis=1)) - z   # Laplacian of the fully connected (weighted) symmetric graph
print(type(z))
print(z.dtype)                   # what is the type() of the entries
print(z)

### convert `list` to `ndarray`
#### ... and an introduction to views

This is also a good opportunity to check attributes like `ndim`, `shape`, `size`, `dtype`, and `itemsize`

In [ ]:
x = list(range(20))
print("x is of type {}".format(type(x)))
print("\nx = {}".format(x))
X = np.array(x)      # casting the list to a numpy.ndarray
print("\nX is of type {}".format(type(X)))
print("\nX =\n{}".format(X))
X.reshape(4, 5)      # reshape does not act on the array, but returns a "view"
print("\nX = \n{}".format(X))
Y = X.reshape(4, 5)  # attribution of a view to a variable stores that view
print("\nY =\n{}".format(Y))
X[3] = 100           # but BE CAREFUL: Y is a reference to a view of X!
print("\nChanged X[3] to {}".format(X[3]))
print("\nY =\n{}".format(Y))

#### can you see what has changed between the version above and below?

In [ ]:
x = list(range(20))
print("x is of type {}".format(type(x)))
print("\nx = {}".format(x))
X = np.array(x)                 # casting the list to a numpy.ndarray
print("\nX is of type {}".format(type(X)))
print("\nX =\n{}".format(X))
X.reshape(4, 5)                 # reshape does not act on the array, but returns a "view"
print("\nX = \n{}".format(X))
Y = X.reshape(4, 5, order='F')  # attribution of a view to a variable stores that view
print("\nY =\n{}".format(Y))
X[3] = 100                      # but BE CAREFUL: Y is a reference to a view of X!
print("\nChanged X[3] to {}".format(X[3]))
print("\nY =\n{}".format(Y))

#### array creation is dynamic (types are derived from the data)
see also [enumerate](https://docs.python.org/3/library/functions.html#enumerate) for `enumerate` or type 
```python
enumerate?
```
in the notebook

In [ ]:
z = list()
z.append(np.array([1, 2, 3]))
z.append(np.array([1., 2, 3]))
z.append(np.array([1 + 1j, 2, 3]))
z.append(np.array(["a", "b", "c"]))

for k, z_ in enumerate(z):
    print("type of elements in z[{ix}] is {x.dtype}".format(ix=k, x=z_))

### Create a `ndarray` from a function

In [ ]:
def f(i, j):
    return np.abs(i - j)

print(np.fromfunction(f, (5, 5), dtype=int))

#### Exercise
Create a matrix that is $n$ by $n$ in size whose entries are
$$
\exp\left(-\frac{\left|x[i] - x[j]\right|^2}{2}\right)
$$
for a vector $x$ of shape $(n,)$.

_Hint_: look for `partial` in the package [functools](https://docs.python.org/3/library/functools.html)

In [ ]:
from functools import partial
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib notebook

n = 50

x = np.random.normal(size=(n,))

# Create the function f that depends on both indices and the data x
# then create g as a partial evaluation of f where the data has been fixed to x

mat = np.fromfunction(g, (n, n), dtype=int)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)
mats = ax.matshow(mat)
plt.colorbar(mats)

## operations on `ndarray`s <a name="operations" />

In [ ]:
def sep(k=5):
    print('\n' + '-'*k + '\n')

x = np.random.randint(201, size=(4, 5))
print(x)
sep()

# universal functions
x -= 100  # inline operator subtracts 100 of each element
print(x)
sep()

# unary functions
print(np.max(x))
print(np.sum(x))
print(x.sum(axis=0))
print(x.sum(axis=0).shape) # /!\ returns a vector, which is always a "column" vector
print(x.sum(axis=1))
print(x.sum(axis=1).shape) # /!\ returns a vector, which is always a "column" vector
sep()

# n-ary functions

print(np.vstack([x] * 3))
sep()
print(np.hstack([x] * 3))
sep()
y = x.T.dot(x)
print(y)

## numpy indexing <a name="indexing" />
[back to index](#index)

In [ ]:
x = np.arange(12).reshape(3, 4, order='F') # --> same as np.array(range(12)).reshape(3, 4)
print(x)
print(x[1, 2])     # single element
print(x[:, 2])     # single column
print(x[[1, 2], [0, 3]]) # two elements

But these are all __views__! 

In [ ]:
y = x[:, 2]
x[0, 2] = 0
print(y)

`ndarray`s can be indexed as lists of lists!

In [ ]:
def sep(n=5):
    print('\n' + n * '-' + '\n')

x = np.random.uniform(size=(2, 3, 4))
print(x)
sep()
print(x[0])
sep()
print(x[0][1])
sep()
print(x[0][1][2])
sep()
print(x[0, 1, 2])

## Copies and views <a name="view" />

In [ ]:
x = np.empty(shape=(2, 3))
print(id(x))
x += 1        # in-place operator --> leaves reference as is
print(id(x))
x = x + 1     # not an in-place operator --> creates a copy
print(id(x))
print(x.base)

In [ ]:
def func(x):
    print("id(x) = {}".format(id(x)))
    print(x.flags.owndata)
    
y = np.empty(shape=(2, 2))
print("id(y) = {}".format(id(y)))
func(y)    # arguments are always passed by reference

In [ ]:
def sep(n=5):
    print('\n' + n * '-' + '\n')
    
y = np.random.randint(11, size=(2, 3)) - 5

w = y
print(w is y)           # True
print(w.base is y)      # False, w is y
print(w.base is y.base) # True, w is y
print(w.flags.owndata)  # True, since y owns data and w is y
sep()

x = y.view()            # create a view only, no ownership
print(x is y)           # False, it is a reference to y
print(x.base is y)      # True, the base reference of x is y
print(x.flags.owndata)  # False, x is not the owner

sep()
z = y.copy()            # create a copy, ownership
print(z is y)           # False
print(z.base is y)      # False, z independent of y
print(z.flags.owndata)  # True, z is the owner

In [ ]:
def sep(n=5):
    print('\n' + n * '-' + '\n')
    
x = np.random.randint(10, size=(2, 3, 4)) # how confusing to use size here !
print(x.size)   # 24 elements
print(x.shape)  # shape (2, 3, 4)
print(x)
sep()

y = x[:, 1, :]  # shape (2, 4)
print(y.shape)
y[0, 0] = -y[0, 0]

print(x)        # y is a reference to a part of x! it's a "view", not a "copy"!

## advanced indexing and broadcasting <a name="broadcasting" />

In [ ]:
def sep(n=5):
    print('\n' + n * '-' + '\n')

x = np.arange(12) * 2
print(x[np.array([1, 1, 3])])                   # [2, 2, 6]
sep()
print(x[np.array([[1, 1], [1, 0]])])            # [[2, 2], [2, 0]]
sep()
print(x.resize(3, 4))                           # resize x to [[0, 2, 4, 6], ...] in-place, returns None!
sep()
print(x)
sep()
print(x[np.array([1, 1]), np.array([0, 3])])    # returns [8, 14]

In [ ]:
x = np.random.normal(size=(1000, 4), loc=10, scale=4)
m = x.mean(axis=0)
s = x.std(axis=0)
print("variables have\n\tmean {loc}\n\t std {scale}".format(loc=m, scale=s))
print(m.shape)
print(s.shape)
print("standardising the variables ...")
x = (x - m) / s            # magic called broadcasting
print("variables have\n\tmean {loc} \n\t std {scale}".format(loc=x.mean(axis=0), scale=x.std(axis=0)))